In [1]:
import time

from chanlun import kcharts
from chanlun.cl_utils import query_cl_chart_config
from chanlun.exchange import get_exchange, Market
from chanlun.exchange.exchange_db import ExchangeDB
from chanlun import cl
from chanlun.xuangu import xuangu
import itertools
import talib
from chanlun.cl_utils import *
from chanlun.cl_interface import Config

market = 'a'
start_date = '2025-01-09 09:31:00'
end_date = '2025-02-21 13:30:00'
frequencys = ['1m']

ex = ExchangeDB(market) # 读取数据库中的k线数据

Chanlun-Pro 缠论数据计算
https://gitee.com/wang-student/chanlun-pro
运行环境: Python: 3.11
授权信息: A_hia_hia_hia-VIP 
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 


In [2]:
zhishu = [
    "SHSE.000300",
]

In [3]:
renders = []
zhishu_klines = {}
for code in zhishu:
    cl_config = query_cl_chart_config(market, code)
    print(cl_config)
    for f in frequencys:
        cl_datas = []
        klines = ex.klines(code, frequency=frequencys, start_date=start_date, end_date=end_date, args={"limit":9999})
        klines = klines[:]
        zhishu_klines[code]=klines
        print(klines[:].head(1))
        print(klines[:].tail(1))
        print(f, '获取K线数据量：', len(klines))
        _s = time.time()
        cl_datas.append(cl.CL(code, f, cl_config).process_klines(klines[:]))
        print('Run time: ', time.time() - _s)

for cd in cl_datas:
    title = '%s - 【%s】 周期数据图表' % (code, cd.frequency)
    renders.append(kcharts.render_charts(title, cd))

{'config_use_type': 'common', 'kline_type': 'kline_default', 'kline_qk': 'none', 'fx_qj': 'fx_qj_ck', 'fx_bh': 'fx_bh_no', 'bi_type': 'bi_type_old', 'bi_bzh': 'bi_bzh_yes', 'bi_qj': 'bi_qj_ck', 'bi_fx_cgd': 'bi_fx_cgd_no', 'xd_qj': 'xd_qj_ck', 'zsd_qj': 'zsd_qj_ck', 'zs_bi_type': ['zs_type_fl'], 'zs_xd_type': ['zs_type_fl'], 'zs_qj': 'zs_qj_ck', 'zs_wzgx': 'zs_wzgx_gd', 'zs_optimize': '0', 'idx_macd_fast': '12', 'idx_macd_slow': '26', 'idx_macd_signal': '9', 'fx_qy': 'fx_qy_middle', 'xd_zs_max_lines_split': '11', 'allow_split_one_line_to_xd': '0', 'allow_bi_fx_strict': '1', 'enable_kchart_low_to_high': '0', 'bi_split_k_cross_nums': '99,0', 'fx_check_k_nums': '99', 'cl_mmd_cal_qs_1mmd': '1', 'cl_mmd_cal_not_qs_3mmd_1mmd': '1', 'cl_mmd_cal_qs_3mmd_1mmd': '1', 'cl_mmd_cal_qs_not_lh_2mmd': '1', 'cl_mmd_cal_qs_bc_2mmd': '1', 'cl_mmd_cal_3mmd_not_lh_bc_2mmd': '1', 'cl_mmd_cal_1mmd_not_lh_2mmd': '1', 'cl_mmd_cal_3mmd_xgxd_not_bc_2mmd': '1', 'cl_mmd_cal_not_in_zs_3mmd': '1', 'cl_mmd_cal_not_in

KeyboardInterrupt: 

In [ ]:
print(klines.columns)

In [ ]:
df = klines
df['close']

In [ ]:
df = calculate_chandelier_exit(klines)
df = calculate_ema(df)

# 可视化
# 创建主坐标轴
fig, ax1 = plt.subplots(figsize=(12, 6))

# 创建右坐标轴
ax2 = ax1.twinx()
delta=480
# 绘制收盘价、长头寸退出点和短头寸退出点
ax1.plot(df['close'][360:360+delta], label='Close Price', color='blue')
ax1.plot(df['EMA90'][360:360+delta], label='EMA90', color='yellow')
ax1.plot(df['Long_Exit'][360:360+delta], label='Chandelier Exit (Long)', linestyle='--', color='green')
ax1.plot(df['Short_Exit'][360:360+delta], label='Chandelier Exit (Short)', linestyle='--', color='red')

# 设置主坐标轴的标签和图例
ax1.set_xlabel('Date')
ax1.set_ylabel('Price', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.legend(loc='upper left')

# 创建右坐标轴
ax2 = ax1.twinx()

# 绘制 Direction 在右坐标轴上
ax2.plot(df['Direction'][360:360+delta], label='Chandelier Exit (Direction)', linestyle=':', color='purple')

# 设置右坐标轴的标签和图例
ax2.set_ylabel('Direction', color='purple')
ax2.tick_params(axis='y', labelcolor='purple')
ax2.legend(loc='upper right')

# 添加标题
plt.title('Chandelier Exit Strategy with Direction on Right Y-Axis')

# 显示图形
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def calculate_chandelier_exit(df, period=22, multiplier=3):
    """
    计算Chandelier Exit指标
    :param df: 包含['high', 'low', 'close']的DataFrame
    :param period: ATR周期 (默认22)
    :param multiplier: AR倍数 (默认3)
    :param direction: 'long' 或 'short'
    :return: 添加指标的DataFrame
    """
    df = df.copy()
    
    # 计算真实波幅TR
    tr1 = df['high'] - df['low']
    tr2 = abs(df['high'] - df['close'].shift())
    tr3 = abs(df['low'] - df['close'].shift())
    df['TR'] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # 计算ATR
    df['ATR'] = df['TR'].rolling(period).mean()
    # 多头止损 = 最高价的period日最高 - ATR×multiplier
    df['CE_high'] = df['high'].rolling(period).max()
    df['Long_Exit'] = df['CE_high'] - multiplier * df['ATR']
    # 空头止损 = 最低价的period日最低 + ATR×multiplier
    df['CE_low'] = df['low'].rolling(period).min()
    df['Short_Exit'] = df['CE_low'] + multiplier * df['ATR']
    df['Direction'] = np.where(df['close'] > df['Long_Exit'], 1, np.where(df['close'] < df['Short_Exit'], -1, 0))

    return df

# 示例数据
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', periods=100)
prices = np.cumsum(np.random.randn(100) * 0.02 + 0.1) + 50
df = pd.DataFrame({
    'date': dates,
    'high': prices + np.random.uniform(0.5, 1.5, 100),
    'low': prices - np.random.uniform(0.5, 1.5, 100),
    'close': prices
}).set_index('date')

# 计算指标
df = calculate_chandelier_exit(df)

# 可视化
plt.figure(figsize=(24, 6))
plt.plot(df['close'], label='Close Price')
plt.plot(df['Long_Exit'], label='Chandelier Exit (Long)', linestyle='--')
plt.plot(df['Short_Exit'], label='Chandelier Exit (Short)', linestyle='--')
plt.plot(df['Direction'], label='Chandelier Exit (Short)', linestyle='--')
plt.title('Chandelier Exit Indicator')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

class ChandelierExit:
    def __init__(self, df, length=22, mult=3.0, use_close=True):
        """
        :param df: 包含OHLC的DataFrame
        :param length: ATR周期
        :param mult: ATR乘数
        :param use_close: 是否使用收盘价计算极值
        """
        self.df = df.copy()
        self.length = length
        self.mult = mult
        self.use_close = use_close
    def calculate_ema(self,ema_period=90):
        # 计算EMA
        self.df[f'EMA{ema_period}'] = self.df['close'].ewm(span=ema_period, adjust=False).mean()
        return self.df
    
    def calculate_atr(self):
        # 计算真实波幅TR
        tr1 = self.df['high'] - self.df['low']
        tr2 = abs(self.df['high'] - self.df['close'].shift(1))
        tr3 = abs(self.df['low'] - self.df['close'].shift(1))
        self.df['TR'] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        # 计算ATR
        self.df['ATR'] = self.df['TR'].rolling(self.length).mean()
        return self.df
    
    def calculate_stops(self):
        # 计算极值
        if self.use_close:
            high_val = self.df['close'].rolling(self.length).max()
            low_val = self.df['close'].rolling(self.length).min()
        else:
            high_val = self.df['high'].rolling(self.length).max()
            low_val = self.df['low'].rolling(self.length).min()
        
        # 初始止损位
        self.df['long_stop'] = high_val - self.mult * self.df['ATR']
        self.df['short_stop'] = low_val + self.mult * self.df['ATR']
        
        # 动态调整止损位
        for i in range(1, len(self.df)):
            prev_long = self.df['long_stop'].iloc[i-1] if not pd.isna(self.df['long_stop'].iloc[i-1]) else self.df['long_stop'].iloc[i]
            prev_short = self.df['short_stop'].iloc[i-1] if not pd.isna(self.df['short_stop'].iloc[i-1]) else self.df['short_stop'].iloc[i]
            
            # 更新多头止损
            if self.df['close'].iloc[i-1] > prev_long:
                self.df['long_stop'].iloc[i] = max(self.df['long_stop'].iloc[i], prev_long)
            else:
                self.df['long_stop'].iloc[i] = self.df['long_stop'].iloc[i]
                
            # 更新空头止损
            if self.df['close'].iloc[i-1] < prev_short:
                self.df['short_stop'].iloc[i] = min(self.df['short_stop'].iloc[i], prev_short)
            else:
                self.df['short_stop'].iloc[i] = self.df['short_stop'].iloc[i]
        
        return self.df
    
    def determine_direction(self):
        # 初始化方向
        self.df['dir'] = 1
        
        for i in range(1, len(self.df)):
            prev_short = self.df['short_stop'].iloc[i-1]
            prev_long = self.df['long_stop'].iloc[i-1]
            
            if self.df['close'].iloc[i] > prev_short:
                self.df['dir'].iloc[i] = 1
            elif self.df['close'].iloc[i] < prev_long:
                self.df['dir'].iloc[i] = -1
            else:
                self.df['dir'].iloc[i] = self.df['dir'].iloc[i-1]
        
        return self.df
    
    def generate_signals(self):
        # 生成买卖信号
        self.df['buy_signal'] = (
                                    (self.df['dir'] == 1) 
                                    & (self.df['dir'].shift(1) == -1) 
                                    & ( self.df['close'] > self.df['EMA90'])
                                )

        
        self.df['sell_signal'] = (
                                    (self.df['dir'] == -1) 
                                    & (self.df['dir'].shift(1) == 1)
                                    & ( self.df['close'] < self.df['EMA90'])
                                )
        return self.df
    
    def run(self):
        self.calculate_ema()
        self.calculate_atr()
        self.calculate_stops()
        self.determine_direction()
        self.generate_signals()
        return self.df

# 使用示例
if __name__ == "__main__":
    # 生成示例数据
    # np.random.seed(42)
    # dates = pd.date_range(start='2023-01-01', periods=100)
    # prices = np.cumsum(np.random.randn(100)*0.02 + 0.1) + 50
    # df = pd.DataFrame({
    #     'date': dates,
    #     'open': prices - np.random.uniform(0.5, 1.5, 100),
    #     'high': prices + np.random.uniform(0.5, 1.5, 100),
    #     'low': prices - np.random.uniform(0.5, 1.5, 100),
    #     'close': prices
    # }).set_index('date')
    
    # 计算指标
    ce = ChandelierExit(klines[5000:], length=22, mult=3.0, use_close=True)
    result = ce.run()
    
    # 可视化
    import matplotlib.pyplot as plt

    # 创建主坐标轴
    fig, ax1 = plt.subplots(figsize=(24, 6))

    # plt.figure(figsize=(12,6))
    ax1.plot(result['close'], label='Close Price')
    ax1.plot(result['EMA90'], label='EMA90')
    ax1.plot(result['long_stop'], label='Long Stop', linestyle='--', color='pink')
    ax1.plot(result['short_stop'], label='Short Stop', linestyle='--', color='pink')

    ax1.legend()
    
    # 创建右坐标轴
    ax2 = ax1.twinx()
    # 绘制 Direction 在右坐标轴上
    ax2.plot(result['dir'], label='Dir', linestyle=':', color='purple')

    # 设置右坐标轴的标签和图例
    ax2.set_ylabel('Dir', color='purple')
    ax2.tick_params(axis='y', labelcolor='purple')
    ax2.legend(loc='upper right')

    ax1.scatter(result[result['buy_signal']].index, 
                result[result['buy_signal']]['close'], 
                marker='^', color='green', s=100, label='Buy Signal', zorder=10)
    ax1.scatter(result[result['sell_signal']].index, 
                result[result['sell_signal']]['close'], 
                marker='v', color='red', s=100, label='Sell Signal', zorder=10)
    
    plt.title('Chandelier Exit Indicator')

    plt.show()


In [ ]:
# 计算CCI指标（以当前标的为例）
def calculate_ema(df, ema_period=90):
    # 计算CCI
    df['EMA90'] = df['close'].ewm(span=ema_period, adjust=False).mean()
    return df

In [ ]:
df = calculate_ema(df)

In [ ]:
print(data[['date','close', 'Long_Exit', 'Short_Exit','EMA90']].tail(15))

In [ ]:
ccis = []
for i in range(len(indicators)):
    cci = calculate_cci(indicators[i])
    cci['CCI_Signal'] = np.where(cci['CCI_EMA'] < -150, 1, 0)
    ccis.append(cci)

In [ ]:
cciss = pd.concat(ccis, axis=1,keys=[f'df_{i}' for i in range(len(ccis))])
cciss

In [ ]:
cciss

In [ ]:
mydata = cciss[('df_0', 'date')]
mydata
print(zhishu)

In [ ]:
mydata = cciss[[('df_0', 'date'),('df_0', 'CCI_Signal'),('df_1', 'CCI_Signal'),('df_2', 'CCI_Signal'),('df_3', 'CCI_Signal'),('df_4', 'CCI_Signal'),('df_5', 'CCI_Signal'),('df_6', 'CCI_Signal'),\
               ('df_7', 'CCI_Signal'),('df_8', 'CCI_Signal'),('df_9', 'CCI_Signal'),\
               ('df_10', 'CCI_Signal'),('df_11', 'CCI_Signal'),('df_12', 'CCI_Signal'),('df_13', 'CCI_Signal'),('df_14', 'CCI_Signal'),('df_15', 'CCI_Signal'),('df_16', 'CCI_Signal')]]

In [ ]:
mydata

In [ ]:
c1 = (cciss[('df_0', 'CCI_Signal')] == 1)
c2 = (cciss[('df_1', 'CCI_Signal')] == 1)
c3 = (cciss[('df_2', 'CCI_Signal')] == 1)

In [ ]:
zzz = cciss[c1 & c2 & c3]
zdate = zzz[("df_0","date")]

In [ ]:
zdate

In [ ]:
# 输出关键数据
print("最新指标值：")
print(all_indicators.iloc[-1])
print("\nCCI信号：", cci['CCI_Signal'].iloc[-1])

In [ ]:
# 可视化示例
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 8))
plt.plot(all_indicators[0], label='上证50指标')
plt.plot(all_indicators[1], label='中盘指标')
plt.plot(all_indicators[2], label='深证成指指标')
plt.title('多指标分析')
plt.legend()
plt.show()

In [ ]:
renders[0]